<div class="alert alert-block alert-info">
LINMA2415 - Project - Question 3

- Colson Adeline 19752000
    
- Lhernould Benoit 77102200 

- El Bared Mary Jo 48212000 
  
    
</div>

<div class="alert alert-block alert-info">
Load Data
</div>

In [1]:
using JuMP
using Gurobi

In [2]:
using CSV
using DataFrames

df = CSV.read("data.csv", DataFrame)
D_t = df[!, "FR_load_forecast_entsoe_transparency"]
println(size(D_t))

(8760,)


<div class="alert alert-block alert-info">
Constants and data
</div>

In [3]:
# Constants and Data
hours_per_year = 8760
lambda = 5000  
cost_of_debt = 0.04
cost_of_equity = 0.07
corporate_tax = 0.30
economic_life = 20


capacity = Dict(
    "Coal" => 1,
    "CCGT" => 1,
    "OCGT" => 1
)

# Technologies data
technologies = Dict(
    "Coal" => (capex=2000*1000, om=0.03, debt_ratio=0.62, heat_rate=2.4, EA=175.2252157*1000, price =5, C=12, emissions=1.4),
    "CCGT" => (capex=950*1000, om=0.03, debt_ratio=0.56, heat_rate=1.62, EA=85.60445144*1000, price =30, C=48.6, emissions=0.5),
    "OCGT" => (capex=700*1000, om=0.03, debt_ratio=0.6, heat_rate=2.5, EA=61.91153839*1000, price =30, C=75, emissions=0.6)
)

Dict{String, NamedTuple{(:capex, :om, :debt_ratio, :heat_rate, :EA, :price, :C, :emissions)}} with 3 entries:
  "Coal" => (capex = 2000000, om = 0.03, debt_ratio = 0.62, heat_rate = 2.4, EA…
  "CCGT" => (capex = 950000, om = 0.03, debt_ratio = 0.56, heat_rate = 1.62, EA…
  "OCGT" => (capex = 700000, om = 0.03, debt_ratio = 0.6, heat_rate = 2.5, EA =…

<div class="alert alert-block alert-info">
Question 3
</div>

In [7]:
model=Model(Gurobi.Optimizer)
@variable(model,0<=p[g in keys(technologies),1:hours_per_year])
@variable(model,0<=k[g in keys(technologies)])
@variable(model,0<=d[1:hours_per_year])
@objective(model,Min,sum((technologies[g].om * technologies[g].capex + technologies[g].EA )*k[g]+sum(technologies[g].C*p[g,i] for i in 1:hours_per_year) for g in keys(technologies))-sum(lambda*d[i] for i in 1:hours_per_year))
@constraint(model,[g in keys(technologies),i in 1:hours_per_year],p[g,i] <= k[g])
@constraint(model,[i in 1:hours_per_year],d[i] <= D_t[i])
@constraint(model,price[i in 1:hours_per_year],d[i]==sum(p[g,i] for g in keys(technologies)))
 
optimize!(model)
    
######################################################################################################################
println("\n")

println("\n")
println("Point I")
println("Optimal capacities (kW):")
total_production = 0
production_by_tech = Dict(g => 0.0 for g in keys(technologies))
for g in keys(technologies)
    cap = value(k[g])  # This ensures 'cap' remains a floating-point number
    println("$(g): $(cap) MW")
    for t in 1:hours_per_year
        production_by_tech[g] += value(p[g, t])  # Accumulate as floating-point values
        total_production += value(p[g, t])  # Keep total production as floating-point
    end
end

println("\n")

println("Point II")
println("Share of each technology in effective production:")
for g in keys(technologies)
    share = production_by_tech[g] / total_production
    println("$(g): $(share * 100) %")
end
println("\n")


println("Point III")
println("Total Production: $(total_production) MWh")
total_cost = sum((technologies[g].EA+technologies[g].om * technologies[g].capex)*value.(k[g])+sum((technologies[g].C)*value.(p[g,i]) for i in 1:hours_per_year)  for g in keys(technologies))
println("Total Investment and Production Cost: €$(total_cost)")
println("\n")


println("Point IV")
# CO2 Emissions data
emissions = Dict(
    "Coal" => 1.4,
    "CCGT" => 0.5,
    "OCGT" => 0.6
)

# Calculating CO2 emissionys
total_co2_emissions = sum(emissions[g] * production_by_tech[g] for g in keys(production_by_tech))
println("\nTotal CO2 Emissions: $(total_co2_emissions) tons")
println("\n")


println("Point V")
load_curtailment = sum(max(0, D_t[t] - value(d[t])) for t in 1:hours_per_year)
println("Total Load Curtailment: $(load_curtailment) MWh")
println("\n")


println("Point VI")
profit = Dict(g => 0.0 for g in keys(technologies))
prices = [-JuMP.dual(price[i]) for i in 1:hours_per_year]

for g in keys(technologies)
    profit[g] = -(technologies[g].EA + technologies[g].om * technologies[g].capex) * value(k[g]) - 
                 sum((technologies[g].C - prices[i]) * value(p[g, i]) for i in 1:hours_per_year)
end

for g in keys(technologies)
    println("Profit for technology $g: $(profit[g])")
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-31
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 43800 rows, 35043 columns and 96360 nonzeros
Model fingerprint: 0xcfb341a3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+04, 1e+05]
Presolve removed 8760 rows and 0 columns
Presolve time: 0.04s
Presolved: 35040 rows, 35043 columns, 87600 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 3
 AA' NZ     : 6.834e+03
 Factor NZ  : 2.506e+04 (roughly 4 MB of memory)
 Factor Ops : 1.435e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective  